<a href="https://colab.research.google.com/github/sdx0112/Chinese-Topic-Modelling/blob/main/ChatGLM2_6B_P_Tuning_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!git clone https://github.com/THUDM/ChatGLM2-6B

Cloning into 'ChatGLM2-6B'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 205 (delta 93), reused 67 (delta 66), pack-reused 87
Receiving objects: 100% (205/205), 7.34 MiB | 5.55 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [2]:
import os
os.chdir('/content/ChatGLM2-6B')

In [3]:
import json

In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 115.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.7 MB/s eta 0:00:

In [5]:
!pip install rouge_chinese nltk jieba datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.9 MB/s eta 0:00:00


In [6]:
import pandas as pd

In [8]:
df = pd.read_csv('/content/all_para.csv', encoding = 'utf_8_sig')
df_sample = pd.read_csv('/content/labelled_sample.csv', encoding = 'GBK')

In [9]:
init_prompt = """你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。
话题列表是：基建, 房地产, 制造业, 科技, 零售消费, 绿色发展, 医药健康, 教育, 交通运输, 出口贸易, 农业农村, 文化旅游, 就业, 民生扶贫, 能源, 物资储备, 宏观, 区域发展, 企业发展, 新冠疫情, 地方财政, 税务, 国际关系, 金融
你的回答的格式必须是：answer-话题,例如：answer-科技
请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。
文字内容是："""

In [10]:
df.head()

,ID,TITLE,CONTENT
0,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,中共中央总书记、国家主席、中央军委主席、中央全面深化改革委员会主任习近平4月21日下午主持召...
1,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议审议通过了《关于强化企业科技创新主体地位的意见》、《关于加强和改进国有经济管理有力支持中...
2,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议指出，强化企业科技创新主体地位，是深化科技体制改革、推动实现高水平科技自立自强的关键举措...
3,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议强调，加强和改进国有经济管理，要立足新时代新征程国有经济肩负的使命任务和功能定位，从服务...
4,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议指出，支持民营经济发展是党中央的一贯方针。促进民营经济发展壮大，要着力优化民营经济发展环...


In [11]:
df_sample.head()

,CONTENT,TOPIC
0,推进基础设施建设和区域协调发展。京津冀协同发展、长江经济带建设取得重要进展。新建铁路投产里程...,基建
1,七是加快建立多主体供应、多渠道保障、租购并举的住房制度。要发展住房租赁市场特别是长期租赁，保...,房地产
2,会议认为，坚持标准引领，建设制造强国，是结构性改革尤其是供给侧结构性改革的重要内容，有利于改...,制造业
3,会议指出，党的十八大以来，党中央部署推进科技评价体系改革，聚焦“四个面向”的科技成果评价导向...,科技
4,会议强调，完善促进消费体制机制，要顺应居民消费新趋势，从供需两端发力，积极培育重点消费领域细...,零售消费


In [12]:
train_json = []
for index, row in df_sample.iterrows():
  train_json.append({'content': init_prompt + row['CONTENT'], 'topic': row['TOPIC']} )

In [13]:
topics_pdf = ['基建', '房地产', '制造业', '科技', '零售消费', '绿色发展', '医药健康', '教育', '交通运输', '出口贸易', '农业农村',
          '文化旅游', '就业', '民生扶贫', '能源', '物资储备', '宏观', '区域发展', '企业发展', '新冠疫情', '地方财政', '税务',
          '国际关系', '金融']

In [14]:
import random

In [20]:
titles = df['TITLE'].unique().tolist()
test_json = [{'content': init_prompt + x, 'topic': random.choice(topics_pdf)} for x in titles]

In [21]:
test_json[:3]

[{'content': '你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。\n话题列表是：基建, 房地产, 制造业, 科技, 零售消费, 绿色发展, 医药健康, 教育, 交通运输, 出口贸易, 农业农村, 文化旅游, 就业, 民生扶贫, 能源, 物资储备, 宏观, 区域发展, 企业发展, 新冠疫情, 地方财政, 税务, 国际关系, 金融\n你的回答的格式必须是：answer-话题,例如：answer-科技\n请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。\n文字内容是：会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代化建设的意见关于促进民营经济发展壮大的意见和中央全面深化改革委员会工作规则中央全面深化改革委员会专项小组工作',
  'topic': '地方财政'},
 {'content': '你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。\n话题列表是：基建, 房地产, 制造业, 科技, 零售消费, 绿色发展, 医药健康, 教育, 交通运输, 出口贸易, 农业农村, 文化旅游, 就业, 民生扶贫, 能源, 物资储备, 宏观, 区域发展, 企业发展, 新冠疫情, 地方财政, 税务, 国际关系, 金融\n你的回答的格式必须是：answer-话题,例如：answer-科技\n请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。\n文字内容是：分析研究当前经济形势和经济工作',
  'topic': '房地产'},
 {'content': '你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。\n话题列表是：基建, 房地产, 制造业, 科技, 零售消费, 绿色发展, 医药健康, 教育, 交通运输, 出口贸易, 农业农村, 文化旅游, 就业, 民生扶贫, 能源, 物资储备, 宏观, 区域发展, 企业发展, 新冠疫情, 地方财政, 税务, 国际关系, 金融\n你的回答的格式必须是：answer-话题,例如：answer-科技\n请给出这段文字的话题

In [17]:
df_val = pd.read_csv('/content/labelled_test.csv', encoding = 'GBK')

In [18]:
val_json = [{'content': init_prompt + row['CONTENT'], 'topic': row['TOPIC']} for index, row in df_val.iterrows()]

In [19]:
val_json[:3]

[{'content': '你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。\n话题列表是：基建, 房地产, 制造业, 科技, 零售消费, 绿色发展, 医药健康, 教育, 交通运输, 出口贸易, 农业农村, 文化旅游, 就业, 民生扶贫, 能源, 物资储备, 宏观, 区域发展, 企业发展, 新冠疫情, 地方财政, 税务, 国际关系, 金融\n你的回答的格式必须是：answer-话题,例如：answer-科技\n请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。\n文字内容是：重大水利工程是“两新一重”建设的重要内容。会议围绕防洪减灾、水资源优化配置、水生态保护修复等，研究了今年及后续150项重大水利工程建设安排，要求抓紧推进建设，促进扩大有效投资，增强防御水旱灾害能力。会议强调，要按照市场化、法治化原则，深化投融资体制机制改革，落实水价标准和收费制度，建立合理回报机制，扩大股权和债权融资规模，以市场化改革推动加快水利工程建设。',
  'topic': '基建'},
 {'content': '你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。\n话题列表是：基建, 房地产, 制造业, 科技, 零售消费, 绿色发展, 医药健康, 教育, 交通运输, 出口贸易, 农业农村, 文化旅游, 就业, 民生扶贫, 能源, 物资储备, 宏观, 区域发展, 企业发展, 新冠疫情, 地方财政, 税务, 国际关系, 金融\n你的回答的格式必须是：answer-话题,例如：answer-科技\n请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。\n文字内容是：会议强调，要有效管控重点风险，守住不发生系统性风险底线。要坚持房子是用来住的、不是用来炒的定位，支持各地从当地实际出发完善房地产政策，支持刚性和改善性住房需求，优化商品房预售资金监管，促进房地产市场平稳健康发展。要及时回应市场关切，稳步推进股票发行注册制改革，积极引入长期投资者，保持资本市场平稳运行。要促进平台经济健康发展，完成平台经济专项整改，实施常态化监管，出台支持平台经济规范健康发展的具体措施',
  't

In [22]:
with open('/content/train.json', 'w') as fout:
    json.dump(train_json , fout)

In [23]:
with open('/content/test.json', 'w') as fout:
    json.dump(test_json , fout)

In [24]:
with open('/content/val.json', 'w') as fout:
    json.dump(val_json , fout)

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoConfig

In [ ]:
model_name = "THUDM/chatglm2-6b"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
tkns = []
for index, row in df_sample.iterrows():
  tkns.append(len(tokenizer(row['CONTENT'])['input_ids']))

In [ ]:
max(tkns)

218

In [ ]:
len(tokenizer(init_prompt)['input_ids'])

169

In [ ]:
169+218

387

In [ ]:
len(init_prompt)

278

In [25]:
os.chdir('/content/ChatGLM2-6B/ptuning')

In [27]:
!bash train.sh

master_addr is only used for static rdzv_backend and when rdzv_endpoint is not specified.
2023-07-14 02:21:33.635887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
07/14/2023 02:21:34 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
07/14/2023 02:21:34 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full

In [28]:
!bash evaluate.sh

master_addr is only used for static rdzv_backend and when rdzv_endpoint is not specified.
2023-07-14 02:34:55.562092: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
07/14/2023 02:34:56 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
07/14/2023 02:34:56 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=True,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full